In [2]:
%pip install kafka-python
%pip install pandas
%pip install numpy
%pip install requests
%pip install pyspark
import json
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from kafka import KafkaConsumer



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
consumer = KafkaConsumer("beehiiv-topic", bootstrap_servers='localhost:9092',
            auto_offset_reset='earliest', enable_auto_commit=True, 
             value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [20]:
import json
from kafka import KafkaConsumer
import time

In [21]:
# Configure Kafka consumer
consumer = KafkaConsumer(
    'beehiiv-topic',
    bootstrap_servers=['localhost:9092'],
    group_id='newsletter-simple-group',
    auto_offset_reset='earliest',
    value_deserializer=lambda x: x.decode('utf-8')
)

In [22]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import psycopg2

In [23]:
#Establishing a connection with the 

conn = psycopg2.connect("host=localhost dbname =NEWSLETTER_DB user=postgres password =0427")
conn.set_session(autocommit = True)
cur= conn.cursor()
#cur.execute("DROP DATABASE IF EXISTS Accounts")
#cur.execute("CREATE DATABASE Accounts")


In [24]:
cur.execute('SET search_path TO NEWSLETTER_SCHEMA;')

In [25]:
try:
    # Query PostgreSQL's information_schema.tables to check for tables
    cur.execute("""
        SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'newsletter_schema';
    """)
    
    tables = cur.fetchall()
    
    if tables:
        print(f"Available tables: {tables}")
    else:
        print("No tables found in the database.")
except Exception as e:
    print(f"Error checking tables: {e}")


Available tables: [('error_log',), ('subscribers',)]


In [26]:
# Process messages from Kafka
for count, message in enumerate(consumer):
    try:
        print(f"\nMessage {count}:")
        print(f"Raw value (first 100 chars): {message.value[:100]}...")

        # Try to parse JSON
        try:
            subscriber_dict = json.loads(message.value)
            print(f"Parsed subscriber: {subscriber_dict.get('email')} (ID: {subscriber_dict.get('subscriber_id')})")
        except json.JSONDecodeError as e:
            print(f"JSON parsing failed: {e}")
            continue  # Skip this message if JSON parsing fails

        # Check if the subscriber exists
        try:
            cur.execute("""
                SELECT COUNT(*) FROM newsletter_schema.subscribers WHERE subscriber_id = %s
            """, (subscriber_dict.get('subscriber_id'),))
            exists = cur.fetchone()[0] > 0

            if exists:
                # Update the subscriber if it already exists
                cur.execute("""
                    UPDATE newsletter_schema.subscribers
                    SET email = %s,
                        name = %s,
                        subscription_status = %s,
                        signup_date = %s,
                        referral_source = %s,
                        tags = %s,
                        ingestion_timestamp = CURRENT_TIMESTAMP
                    WHERE subscriber_id = %s
                """, (
                    subscriber_dict.get('email'),
                    subscriber_dict.get('name'),
                    subscriber_dict.get('subscription_status'),
                    subscriber_dict.get('signup_date'),
                    subscriber_dict.get('referral_source'),
                    subscriber_dict.get('tags'),
                    subscriber_dict.get('subscriber_id')
                ))

                print(f"Successfully updated subscriber (ID: {subscriber_dict.get('subscriber_id')})")

            else:
                # Insert the subscriber if it doesn't exist
                cur.execute("""
                    INSERT INTO newsletter_schema.subscribers (subscriber_id, email, name, subscription_status, signup_date, referral_source, tags)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                """, (
                    subscriber_dict.get('subscriber_id'),
                    subscriber_dict.get('email'),
                    subscriber_dict.get('name'),
                    subscriber_dict.get('subscription_status'),
                    subscriber_dict.get('signup_date'),
                    subscriber_dict.get('referral_source'),
                    subscriber_dict.get('tags')
                ))

                print(f"Successfully inserted new subscriber (ID: {subscriber_dict.get('subscriber_id')})")

            # Commit after each insert/update
            conn.commit()

        except Exception as db_error:
            print(f"Database error: {db_error}")
            # Log error to PostgreSQL error log table
            try:
                cur.execute("""
                    INSERT INTO newsletter_schema.error_log (error_message)
                    VALUES (%s)
                """, (str(db_error),))
                conn.commit()
            except Exception as log_error:
                print(f"Error logging to PostgreSQL: {log_error}")

    except Exception as e:
        print(f"Error processing message: {e}")
        continue  # Skip this message if any error occurs

print("Finished processing messages")

# Close connections
cur.close()
conn.close()
consumer.close()


Message 0:
Raw value (first 100 chars): {"subscriber_id": "914001c1-156a-4af2-a927-48fb798ffd79", "email": "llucas@example.net", "name": NaN...
Parsed subscriber: llucas@example.net (ID: 914001c1-156a-4af2-a927-48fb798ffd79)
Successfully inserted new subscriber (ID: 914001c1-156a-4af2-a927-48fb798ffd79)

Message 1:
Raw value (first 100 chars): {"subscriber_id": "20a20baa-bdc8-49b2-ac29-3509dc2bc883", "email": "millerstephanie@example.net", "n...
Parsed subscriber: millerstephanie@example.net (ID: 20a20baa-bdc8-49b2-ac29-3509dc2bc883)
Successfully inserted new subscriber (ID: 20a20baa-bdc8-49b2-ac29-3509dc2bc883)

Message 2:
Raw value (first 100 chars): {"subscriber_id": "d7d3fb2b-8a7d-4000-a654-2ad766f653b8", "email": "nrussell@example.org", "name": "...
Parsed subscriber: nrussell@example.org (ID: d7d3fb2b-8a7d-4000-a654-2ad766f653b8)
Successfully inserted new subscriber (ID: d7d3fb2b-8a7d-4000-a654-2ad766f653b8)

Message 3:
Raw value (first 100 chars): {"subscriber_id": "b318de61-6e

KeyboardInterrupt: 

In [27]:
cur.close()
conn.close()
consumer.close()